## Susipažinimas
Iš pradžių nebuvau tikras, kokie duomenys yra .tiff failuose, todėl nusprendžiau išsiaiškinti nelabai efektyviais būdais. Mažesnį failą nuskaitė per maždaug 10 sekundžių.

In [1]:
from tifffile import tifffile
from PIL import Image
import pptk
import numpy as np
from time import time

In [2]:
t = time()
im = tifffile.imread('paseliai/clf_LIT_2021_ET100_18pas.tif')
print(f'Failo nuskaitymo laikas:', round(time()-t, 3))
print(f'Matmenys: {im.shape}')
print(f'Duomenų tipas: {im.dtype}')
print(f'min: {np.min(im)}', f'max: {np.max(im)}')

Failo nuskaitymo laikas: 10.684
Matmenys: (31000, 38000)
Duomenų tipas: uint16
min: 0 max: 601


In [26]:
im

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

Ok, tik ką reiškia tie skaičiai? Galbūt tai kažkas panašaus į reljefo aukštį? Pasižiūrim 3D vaizdą. Tam parašiau gudrią komandą, dvimačio masyvo taškus paverčiančią į taškus, reikalingus 3D atvaizdavimui.

In [28]:
def gudri_komanda(arr):
    return np.swapaxes(np.array([*np.indices(arr.shape), arr]), 0, 2).reshape(-1,3)

Tuomet pasinaudojau atvaizdavimo įrankiu [`pptk`](https://heremaps.github.io/pptk/viewer.html), skirtu vaizduoti dideliems taškų debesims

In [35]:
small_image = im[-4000:, 16000:20000] #Paimu fragmentą su 16M taškų
xyz = gudri_komanda(small_image)
xyz = xyz[xyz[:,2] != 0] #vaizduosiu tik taškus su nenulinėmis z koordinatėmis; jų yra apie 0.3M
v = pptk.viewer(xyz)
v.attributes(np.ones(xyz.shape))
v.set(point_size=0.1)

Gavau tokį vaizdą:

![](3D_points.PNG)

Po kurio laiko supratau, kad kiekvienam taškui yra priskirtas sklypo ID, kuris dažniausiai būna 0. Kai jau viskas aišku, galima pereiti prie skaičiavimo, koks kievieno ID pasikartojimas.

## Skaičiavimas ir jo optimizacija
Kievieną skirtingą skaičiavimo algoritmą leidžiu ant $31000\times 38000$ masyvo `im`, nuskaityto iš 99MB failo `clf_LIT_2021_ET100_18pas.tif`. 

Expected output:

    {0: 934025302, 101: 11456189, 102: 6028877, 104: 4722672, 105: 4393260, 106: 13221637, 107: 70603190, 108: 12680311, 109: 2673109, 110: 1666446, 111: 29668404, 113: 3387695, 153: 6455884, 154: 1170747, 161: 47393, 201: 5414660, 203: 7592068, 205: 142835, 601: 62649321}

### `np.unique`, 2D case
Vienas dažniausiai naudojamų metodų `np.unique`, taikomas dvimačiam masyvui 

In [46]:
def unique_2D_numpy(a):
    u, counts = np.unique(a, return_counts=True) #viename masyve ID, kitame jų dažnumai
    return dict(zip(u, counts))

In [47]:
%%timeit -r 4 -n 1
out = unique_2D_numpy(im)

1min 18s ± 2.02 s per loop (mean ± std. dev. of 4 runs, 1 loop each)


### `np.unique`, 1D case
Metodas `np.unique`, taikomas prieš tai paverčiant masyvą į vienmatį

In [52]:
def unique_1D_numpy(a):
    u, counts = np.unique(a.ravel(), return_counts=True) #viename masyve ID, kitame jų dažnumai
    return dict(zip(u, counts))

In [55]:
%%timeit -r 4 -n 1
out = unique_1D_numpy(im)

1min 21s ± 4.33 s per loop (mean ± std. dev. of 4 runs, 1 loop each)


Išvada: `np.unique` veikia vienodai lėtai ant to paties dydžio 1D ir 2D masyvų.

### `np.bincount`
Metodas `np.bincount` veikia efektyviau už `np.unique`, kai yra nedidelė sveikųjų ID amplitudė. Tai patvirtina ir [šių matavimų rezultatai](https://stackoverflow.com/a/43096495/3044825). Jis grąžina skaičių nuo 0 iki `np.max(im)` dažnių sąrašą, iš kurio vėliau reikia išrinkti tik tuos, kurių dažnis nenulinis. **Svarbu: negalima `np.bincount` leisti ant didelių masyvų, nes viršijamas RAM, nuteka atmintis ir tai stipriai atsiliepia veikimo trukmei**. Todėl būtina skaidyti `im` į atskirus dvimačius masyvus ir sumuoti tarpinius skaičiavimų rezultatus. Prieš leidžiant `np.bincount` ant dvimačių masyvų svarbu juos prieš tai suplokštinti.

Šiek tiek laiko galima sutaupyti pašalinant nulinius elementus naudojant `np.flatnonzero` metodą, grąžinantį nenulinių elementų indeksus. Tai daug efektyviau už nulinių elementų skaičiavimą `np.bincount` viduje. Vėliau nulių dažnis yra sužinomas atimant likusius dažnius iš `im` dydžio. 

`im` turi ~79% nulių, procesinimas pagreitėja ~21%.

In [7]:
from collections import Counter

def bincount_2D_numpy(arr, chunk_size = 1000, speed_up=False):
    def bincount(a):
        y = np.bincount(a)
        ii = np.nonzero(y)[0]
        return dict(zip(ii, y[ii]))
    
    c = Counter({})
    for s1 in range(0, arr.shape[0], chunk_size):
        for s2 in range(0, arr.shape[1], chunk_size):
            chunk = arr[s1:s1+chunk_size, s2:s2+chunk_size].ravel() #2D masyvas paverčiamas į 1D
            if speed_up:
                cnt = bincount(chunk[np.flatnonzero(chunk)])
            else:
                cnt = bincount(chunk)
            c = c + Counter(cnt)
    out = dict(c)
    if speed_up:
        return {**{0:arr.size - sum(out.values())},**out}
    else:
        return out

In [62]:
%%timeit -r 7 -n 3
out = bincount_2D_numpy(im, speed_up=False)

17 s ± 846 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [71]:
%%timeit -r 7 -n 3
out = bincount_2D_numpy(im, speed_up=True)

13.4 s ± 350 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


### Let's try numba
[Some clues about it](https://stackoverflow.com/questions/46256279/bin-elements-per-row-vectorized-2d-bincount-for-numpy)

Trivial case:

In [41]:
from numba import njit, prange

def bincount_2D_numba(a, use_parallel=False, use_prange=False):
    m, n = a.shape
    counts = np.zeros(a.max()+1, dtype=int)
    # Choose function based on args
    func = bincount2D_numba_func0
    if use_parallel:
        if use_prange:
            func = bincount2D_numba_func2
        else:
            func = bincount2D_numba_func1
    # Run chosen function on input data and output
    func(a, counts, m, n)    
    ii = np.flatnonzero(counts)
    return dict(zip(ii, counts[ii]))

@njit
def bincount2D_numba_func0(a, counts, m, n):
    for i in range(m):
        for j in range(n):
            counts[a[i,j]] += 1

@njit(parallel=True)
def bincount2D_numba_func1(a, counts, m, n):
    for i in range(m):
        for j in range(n):
            counts[a[i,j]] += 1

@njit(parallel=True)
def bincount2D_numba_func2(a, counts, m, n):
    for i in prange(m):
        for j in prange(n):
            counts[a[i,j]] += 1

In [42]:
from numba import njit, prange

def bincount_1D_numba(a, use_parallel=False, use_prange=False):
    m = len(a)
    counts = np.zeros(a.max()+1, dtype=int)
    # Choose function based on args
    func = bincount1D_numba_func0
    if use_parallel:
        if use_prange:
            func = bincount1D_numba_func2
        else:
            func = bincount1D_numba_func1
    # Run chosen function on input data and output
    func(a, counts, m)    
    ii = np.flatnonzero(counts)
    return dict(zip(ii, counts[ii]))

@njit
def bincount1D_numba_func0(a, counts, m):
    for i in range(m):
        counts[a[i]] += 1

@njit(parallel=True)
def bincount1D_numba_func1(a, counts, m):
    for i in range(m):
        counts[a[i]] += 1

@njit(parallel=True)
def bincount1D_numba_func2(a, counts, m):
    for i in prange(m):
        counts[a[i]] += 1

In [43]:
def bincount_numba(arr, chunk_size = 10000, use_parallel=False, use_prange=False, ravel=False):
    z = 0
    c = Counter({})
    for s1 in range(0, arr.shape[0], chunk_size):
        for s2 in range(0, arr.shape[1], chunk_size):
            chunk = arr[s1:s1+chunk_size, s2:s2+chunk_size]
            if ravel:
                cnt = bincount_1D_numba(chunk.ravel(), use_parallel=use_parallel, use_prange=use_prange)
            else:
                cnt = bincount_2D_numba(chunk, use_parallel=use_parallel, use_prange=use_prange)           
            c = c + Counter(cnt)
    out = dict(c)
    return out

In [45]:
%%timeit -r 1 -n 1
out = bincount_numba(im, use_parallel=False, use_prange=False, ravel=False)

6.22 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Reading

[Might large tiff?](https://gis.stackexchange.com/questions/268439/processing-large-geotiff-using-python/268452)  
[More conqrete](https://gis.stackexchange.com/questions/402202/how-to-compress-split-a-tiff-file-4gb-in-order-to-work-with-it-in-python)  
[GDAL installation](https://opensourceoptions.com/blog/how-to-install-gdal-with-anaconda/)  
[Doing ReadAsArray fast](https://stackoverflow.com/questions/41742162/gdal-readasarray-for-vrt-extremely-slow)

In [31]:
from time import time
from osgeo import gdal

def read_raster(path):
    ds = gdal.OpenEx(path, gdal.GA_ReadOnly) #raster_dataset
    gdal.AllRegister()
    band = ds.GetRasterBand(1)
    w1, w2 = band.GetBlockSize()
    r1, r2 = range(0, ds.RasterXSize, w1), range(0, ds.RasterYSize, w2)
    print(f'r1 = {r1}, r2 = {r2}, (w1, w2) = ({w1}, {w2})')
    for i in r1:
        for j in r2:
            if j%1000==0: print(j, end=' ')
            data = band.ReadAsArray(i, j, w1, w2) #x,y,xw,yw
            #return data

In [ ]:
%%timeit -r 1 -n 1
#read_raster("paseliai/clf_LIT_2021_ET100_18pas.tif")
data = read_raster("paseliai/paseliai_2020_sklypo_ID.tif")

r1 = range(0, 37942, 37942), r2 = range(0, 28659), (w1, w2) = (37942, 1)
0 1000 2000 3000 4000 5000 6000 7000 8000 

In [29]:
path = "paseliai/clf_LIT_2021_ET100_18pas.tif"
ds = gdal.OpenEx(path, gdal.GA_ReadOnly) #raster_dataset
band = ds.GetRasterBand(1)
band, ds.GetGeoTransform()

(<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x000001DDCEF05960> >,
 (5000000.0, 10.0, 0.0, 3810000.0, 0.0, -10.0))

In [30]:
path = "paseliai/paseliai_2020_sklypo_ID.tif"
ds = gdal.OpenEx(path, gdal.GA_ReadOnly) #raster_dataset
band = ds.GetRasterBand(1)
band, ds.GetGeoTransform()

(<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x000001DDCEEF2510> >,
 (301730.0, 10.0, 0.0, 6258830.0, 0.0, -10.0))

In [56]:
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
geotransform = raster_dataset.GetGeoTransform() #6 items
#top left x; w; rotation, 0 if image is "north up"; 
#top left y; rotation, 0 if image is "north up"; n-s pixel resolution

originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]